<a href="https://colab.research.google.com/github/danieltalon/trabalho_RAG/blob/main/3_verifica_respostas_taxa_acerto_IA_NAIVE_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# -*- coding: utf-8 -*-
"""
Avaliação de IA em Questões Médicas (Adaptado para Google Colab - COM RAG de Múltiplos PDFs, Persistência de Índice, BATCH API e Log Timestamped)

Este script avalia o desempenho de um modelo de IA generativa (Google Gemini)
em responder questões de múltipla escolha de provas de medicina,
filtradas por áreas específicas. Utiliza RAG com múltiplos PDFs de um diretório,
salva/carrega o índice vetorial FAISS e os chunks de texto para persistência,
envia perguntas em lotes para a API de geração e salva resultados e logs com timestamp.
"""

# @title 1. Instalar Bibliotecas e Importar Módulos Necessários
# Instala as bibliotecas necessárias no ambiente Colab
!pip install -q google-generativeai pandas pypdf2 faiss-cpu numpy

import json
import csv
import time
import re
import pandas as pd
import numpy as np
import google.generativeai as genai
from google.colab import userdata, drive, files # Módulos específicos do Colab
from collections import defaultdict
import io
from PyPDF2 import PdfReader # Para ler o PDF
import faiss # Para o banco de dados vetorial
import os # Para listar arquivos e verificar existência
import pickle # Para salvar/carregar a lista de chunks
from datetime import datetime # Para timestamp

# @title 2. Definir Constantes e Configurações

# --- Constantes de Configuração ---
# Caminhos atualizados conforme sua especificação
JSON_FILE_PATH = '/content/drive/MyDrive/Questoes/questoes_2021_2022_2023_2024_limit700_classificadas_lote.json' #@param {type:"string"}
RESULTS_DIR = '/content/drive/MyDrive/Resultados/' #@param {type:"string"} # Diretório para salvar CSV e Log
BASE_CSV_FILENAME = 'resultados_avaliacao_ia_naive_rag_lote' #@param {type:"string"}
BASE_LOG_FILENAME = 'log_avaliacao_ia_naive_rag_lote' #@param {type:"string"}
AREAS_TO_FILTER = ["Ginecologia", "Obstetrícia", "Pediatria"] #@param

# --- Constantes RAG ---
KNOWLEDGE_BASE_DIR = '/content/drive/MyDrive/Knowledge_Base/' #@param {type:"string"} # Diretório com os PDFs
INDEXES_DIR = '/content/drive/MyDrive/Indexes/' #@param {type:"string"} # Diretório para salvar/carregar índices
FAISS_INDEX_PATH = os.path.join(INDEXES_DIR, 'medicina_knowledge.index') # Nome do arquivo do índice
TEXT_CHUNKS_PATH = os.path.join(INDEXES_DIR, 'medicina_knowledge_chunks.pkl') # Nome do arquivo dos chunks

CHUNK_SIZE = 1000 #@param {type:"integer"}
CHUNK_OVERLAP = 100 #@param {type:"integer"}
TOP_K_RAG = 3 #@param {type:"integer"}
EMBEDDING_MODEL = 'models/text-embedding-004' #@param {type:"string"}

# --- Parâmetros de Processamento ---
BATCH_SIZE = 15 #@param {type:"integer"}
DELAY_BETWEEN_BATCHES = 3 #@param {type:"integer"}
MAX_RETRIES = 3 #@param {type:"integer"}
MAX_QUESTIONS_TO_PROCESS = 150 #@param {type:"integer"}
GEMINI_MODEL_NAME = 'gemini-1.5-flash' #@param {type:"string"}
COLAB_SECRET_KEY_NAME = 'GOOGLE_API_KEY' #@param {type:"string"}
AI_TEMPERATURE = 0.1 #@param {type:"number"}

#-------------------------------------

# @title 3. Montar Google Drive e Criar Diretórios
try:
    drive.mount('/content/drive')
    print("Google Drive montado com sucesso!")
    # Cria os diretórios de índices e resultados se não existirem
    os.makedirs(INDEXES_DIR, exist_ok=True)
    os.makedirs(RESULTS_DIR, exist_ok=True)
    print(f"Diretório de índices '{INDEXES_DIR}' verificado/criado.")
    print(f"Diretório de resultados '{RESULTS_DIR}' verificado/criado.")
except Exception as e:
    print(f"Erro ao montar Google Drive ou criar diretórios: {e}")
    # exit()

# @title 4. Configurar API Key do Google
try:
    API_KEY = userdata.get(COLAB_SECRET_KEY_NAME)
    if not API_KEY:
        raise ValueError(f"Chave secreta '{COLAB_SECRET_KEY_NAME}' não encontrada ou vazia.")
    genai.configure(api_key=API_KEY)
    print("API Key do Google configurada com sucesso via Colab Secrets.")
except Exception as e:
    print(f"Erro ao configurar a API Key: {e}")
    # exit()

# @title 5. Funções Auxiliares (Carregar, Filtrar, RAG, Formatar, Chamar IA, Parsear, Avaliar, Salvar)

# --- Funções de Carregamento e Filtragem (sem alterações) ---
def load_json_data(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Erro: Arquivo JSON não encontrado em '{filepath}'.")
        return None
    except json.JSONDecodeError:
        print(f"Erro: Falha ao decodificar o arquivo JSON '{filepath}'")
        return None
    except Exception as e:
        print(f"Erro inesperado ao carregar o JSON: {e}")
        return None

def filter_questions(data, areas):
    filtered = []
    if not data: return filtered
    seen_ids = set()
    for i, question in enumerate(data):
        prova = question.get('prova', 'unk_prova'); numero = question.get('numero', f'idx_{i}')
        question_id = f"{prova}_{numero}"
        if question_id in seen_ids: continue
        question_areas = question.get('areas_medicas', [])
        if any(area in question_areas for area in areas):
            if 'enunciado' in question and 'alternativas' in question and 'resposta' in question:
                if isinstance(question['alternativas'], dict):
                    filtered.append({
                        'id': question_id, 'enunciado': question['enunciado'],
                        'alternativas': question['alternativas'],
                        'resposta_correta': str(question['resposta']).strip().upper(),
                        'areas_medicas': question_areas,
                        'continha_imagem': question.get('contains_img', False)
                    })
                    seen_ids.add(question_id)
                else: print(f"Aviso: Alternativas inválidas ID {question_id}. Pulando.")
            else: print(f"Aviso: Questão ID {question_id} faltando campos. Pulando.")
    return filtered

# --- Funções RAG (Modificada para carregar múltiplos PDFs e persistência) ---

def load_and_split_pdfs_from_directory(directory_path, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP):
    """Carrega TODOS os PDFs de um diretório, extrai texto e divide em chunks."""
    print(f"Carregando e processando PDFs do diretório: {directory_path}")
    all_text_chunks = []
    pdf_files_processed = [] # Lista para guardar nomes dos arquivos processados

    try:
        items_in_dir = os.listdir(directory_path)
        pdf_files_found = [os.path.join(directory_path, item) for item in items_in_dir
                           if os.path.isfile(os.path.join(directory_path, item)) and item.lower().endswith('.pdf')]

        if not pdf_files_found:
            print(f"Nenhum arquivo PDF encontrado diretamente em '{directory_path}'.")
            return None, [] # Retorna None para chunks e lista vazia de arquivos

        print(f"Arquivos PDF encontrados: {len(pdf_files_found)}")

        for pdf_path in pdf_files_found:
            pdf_filename = os.path.basename(pdf_path)
            print(f"  Processando: {pdf_filename}...")
            try:
                reader = PdfReader(pdf_path)
                pdf_full_text = ""
                for page_num, page in enumerate(reader.pages):
                    try: pdf_full_text += page.extract_text() + "\n"
                    except Exception as page_error: print(f"    Aviso: Erro ao extrair página {page_num + 1} de {pdf_filename}: {page_error}"); pdf_full_text += "\n[Erro na Página]\n"
                start_index = 0; pdf_chunks_count = 0
                while start_index < len(pdf_full_text):
                    end_index = min(start_index + chunk_size, len(pdf_full_text))
                    chunk_text = pdf_full_text[start_index:end_index]
                    if chunk_text.strip(): all_text_chunks.append(chunk_text); pdf_chunks_count += 1
                    next_start = start_index + chunk_size - chunk_overlap
                    if next_start <= start_index: next_start = start_index + 1
                    start_index = next_start
                    if end_index == len(pdf_full_text): break
                print(f"    -> {pdf_chunks_count} chunks gerados.")
                pdf_files_processed.append(pdf_filename) # Adiciona à lista de processados

            except Exception as file_error: print(f"  Erro ao processar PDF '{pdf_filename}': {file_error}. Pulando."); continue

        print(f"Total de chunks de todos os PDFs processados: {len(all_text_chunks)}")
        return all_text_chunks, pdf_files_processed # Retorna chunks e nomes dos arquivos

    except FileNotFoundError: print(f"Erro: Diretório '{directory_path}' não encontrado."); return None, []
    except Exception as e: print(f"Erro inesperado ao listar/processar PDFs: {e}"); return None, []


def generate_embeddings(text_chunks, model_name=EMBEDDING_MODEL):
    # (Função sem alterações)
    if not text_chunks: return None
    print(f"Gerando embeddings para {len(text_chunks)} chunks usando {model_name}...")
    embeddings = []
    count = 0
    total = len(text_chunks)
    embedding_batch_size = 100
    for i in range(0, total, embedding_batch_size):
        batch_chunks = text_chunks[i:min(i + embedding_batch_size, total)]
        try:
            result = genai.embed_content(model=model_name, content=batch_chunks, task_type="retrieval_document")
            embeddings.extend(result['embedding'])
            count += len(batch_chunks)
            print(f"  Embeddings gerados para {count}/{total} chunks...")
            time.sleep(1)
        except Exception as e:
            print(f"Erro ao gerar embeddings para o lote {i//embedding_batch_size + 1}: {e}. Tentando individualmente...")
            for single_chunk in batch_chunks:
                 try:
                     result = genai.embed_content(model=model_name, content=single_chunk, task_type="retrieval_document")
                     embeddings.append(result['embedding'])
                     count += 1
                     if count % 50 == 0: print(f"  Embeddings gerados para {count}/{total} chunks (modo individual)...")
                     time.sleep(0.2)
                 except Exception as e_ind:
                     print(f"Erro ao gerar embedding para chunk individual: {e_ind}. Adicionando vetor nulo.")
                     embeddings.append([0.0] * 768)
                     time.sleep(1)
            count = min(i + embedding_batch_size, total)
    if len(embeddings) != total: print(f"Aviso: Número de embeddings ({len(embeddings)}) diferente do número de chunks ({total}).")
    print("Geração de embeddings concluída.")
    return np.array(embeddings).astype('float32')

def create_faiss_index(embeddings):
    # (Função sem alterações)
    if embeddings is None or len(embeddings) == 0: print("Erro: Nenhum embedding para criar o índice."); return None
    try:
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)
        print(f"Índice FAISS criado com {index.ntotal} vetores de dimensão {dimension}.")
        return index
    except Exception as e: print(f"Erro ao criar índice FAISS: {e}"); return None

def retrieve_relevant_context(query, index, text_chunks, embedding_model=EMBEDDING_MODEL, top_k=TOP_K_RAG):
    # (Função sem alterações)
    if index is None or not text_chunks: return "Erro: Índice ou chunks não disponíveis."
    try:
        query_embedding_result = genai.embed_content(model=embedding_model, content=query, task_type="retrieval_query")
        query_embedding = np.array([query_embedding_result['embedding']]).astype('float32')
        distances, indices = index.search(query_embedding, top_k)
        relevant_chunks = [text_chunks[idx] for idx in indices[0] if 0 <= idx < len(text_chunks)]
        return "\n---\n".join(relevant_chunks)
    except Exception as e: print(f"Erro durante a recuperação RAG para query '{query[:50]}...': {e}"); return "Erro na recuperação de contexto."

# --- Funções de Formatação, Chamada e Parsing da IA (sem alterações da versão RAG anterior) ---
def format_augmented_batch_for_ai(batch_questions_with_context):
    prompt = f"Responda às seguintes questões de múltipla escolha usando o contexto fornecido para cada uma. Se o contexto não for suficiente, use seu conhecimento geral. Para cada questão, forneça APENAS a letra da alternativa correta, precedida pelo número da questão no lote e um ponto (ex: 1. A, 2. C, 3. B).\n\n"
    for i, data in enumerate(batch_questions_with_context):
        question_data = data['question']; context = data['context']
        prompt += f"--- Questão {i+1} ---\nContexto Recuperado:\n{context}\n---\n"
        prompt += f"Enunciado: {question_data['enunciado']}\n\nAlternativas:\n"
        alternatives = question_data['alternativas']
        try: sorted_keys = sorted(alternatives.keys())
        except TypeError: sorted_keys = alternatives.keys()
        for key in sorted_keys: prompt += f"{key}: {alternatives[key]}\n"
        if question_data['continha_imagem']: prompt += "(Obs: Imagem não exibida)\n"
        prompt += "\n"
    prompt += "Respostas (formato: 1. Letra, 2. Letra, ...):\n"
    return prompt

def parse_ai_batch_response(response_text, batch_size):
    answers = {}
    if not response_text:
        print("Aviso: Resposta da IA para o lote está vazia.")
        for i in range(batch_size): answers[i] = "Erro: Resposta Vazia no Lote"
        return [answers.get(i, "Erro: Resposta Ausente no Lote") for i in range(batch_size)]
    cleaned_text = re.sub(r'[*`]', '', response_text).strip()
    pattern = re.compile(r"^\s*(\d+)\s*\.\s*([A-E])\b", re.MULTILINE)
    found_answers = pattern.findall(cleaned_text.upper())
    if found_answers:
        for num_str, letter in found_answers:
            try:
                question_index = int(num_str) - 1
                if 0 <= question_index < batch_size:
                    if question_index not in answers: answers[question_index] = letter
            except ValueError: print(f"Aviso: Não foi possível converter número '{num_str}'.")
    else:
        lines = cleaned_text.split('\n')
        potential_answers = [line.strip() for line in lines if len(line.strip()) == 1 and 'A' <= line.strip().upper() <= 'E']
        if len(potential_answers) == batch_size:
             print("Aviso: Usando formato alternativo de parsing (uma letra por linha).")
             for i, ans in enumerate(potential_answers): answers[i] = ans.upper()
        else: print(f"Aviso: Não foi possível parsear resposta do lote. Resposta:\n{cleaned_text}")
    answer_list = [answers.get(i, "Erro de Parsing no Lote") for i in range(batch_size)]
    return answer_list

def get_ai_answers_for_augmented_batch(augmented_prompt, batch_size, model_name=GEMINI_MODEL_NAME, temperature=AI_TEMPERATURE, max_retries=MAX_RETRIES):
    model = genai.GenerativeModel(model_name)
    attempts = 0
    generation_config = genai.types.GenerationConfig(temperature=temperature)
    safety_settings = [{"category": c, "threshold": "BLOCK_NONE"} for c in ["HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH", "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT"]]
    while attempts < max_retries:
        try:
            response = model.generate_content(augmented_prompt, generation_config=generation_config, safety_settings=safety_settings)
            if not response.parts:
                 block_reason = response.prompt_feedback.block_reason if hasattr(response, 'prompt_feedback') and response.prompt_feedback else "Desconhecida"
                 finish_reason = response.candidates[0].finish_reason if hasattr(response, 'candidates') and response.candidates else "Desconhecido"
                 if finish_reason != 'STOP':
                     print(f"Aviso: Resposta bloqueada LOTE RAG. Razão: {block_reason} / Finish: {finish_reason}")
                     return [f"Bloqueado ({block_reason})" for _ in range(batch_size)]
                 else:
                     print("Aviso: Resposta vazia LOTE RAG (Finish Reason STOP).")
                     return ["Erro: Resposta Vazia no Lote" for _ in range(batch_size)]
            ai_answers_raw = response.text
            parsed_answers = parse_ai_batch_response(ai_answers_raw, batch_size)
            return parsed_answers
        except Exception as e:
            attempts += 1; print(f"Erro API Gemini LOTE RAG (Tentativa {attempts}/{max_retries}): {e}")
            wait_time = 2 ** attempts
            if "429" in str(e) or "resource_exhausted" in str(e).lower(): wait_time = max(wait_time, 10); print(f"Rate limit/Recurso Esgotado. Aguardando {wait_time}s...")
            elif attempts < max_retries: print(f"  Aguardando {wait_time}s...")
            else: print(f"Falha LOTE RAG após {max_retries} tentativas."); return ["Erro de API" for _ in range(batch_size)]
            time.sleep(wait_time)
    return ["Erro: Máximo de tentativas atingido" for _ in range(batch_size)]

# --- Função Principal de Avaliação (sem alterações da versão RAG anterior) ---
def evaluate_questions_with_rag(questions_to_process, faiss_index, text_chunks):
    results = []
    total_questions = len(questions_to_process)
    print(f"Iniciando avaliação RAG de {total_questions} questões em lotes de {BATCH_SIZE}...")
    if faiss_index is None or not text_chunks:
        print("ERRO: Índice FAISS ou chunks não disponíveis. Abortando avaliação RAG.")
        return results
    for i in range(0, total_questions, BATCH_SIZE):
        batch_questions_data = questions_to_process[i:min(i + BATCH_SIZE, total_questions)]
        current_batch_size = len(batch_questions_data)
        print(f"\n--- Processando Lote RAG {i // BATCH_SIZE + 1}/{(total_questions + BATCH_SIZE - 1) // BATCH_SIZE} (Questões {i + 1} a {i + current_batch_size}) ---")
        batch_questions_with_context = []
        print("  Recuperando contexto RAG para o lote...")
        for q_idx, question_data in enumerate(batch_questions_data):
            print(f"    Recuperando para questão {i+1+q_idx} (ID: {question_data['id']})...")
            query = question_data['enunciado']
            retrieved_context = retrieve_relevant_context(query, faiss_index, text_chunks)
            batch_questions_with_context.append({'question': question_data, 'context': retrieved_context})
            time.sleep(0.1)
        print("  Formatando prompt aumentado para a IA...")
        augmented_batch_prompt = format_augmented_batch_for_ai(batch_questions_with_context)
        print("  Enviando lote aumentado para a IA...")
        ai_answers_for_batch = get_ai_answers_for_augmented_batch(augmented_batch_prompt, current_batch_size)
        print("  Processando respostas do lote...")
        for j, augmented_data in enumerate(batch_questions_with_context):
            question_data = augmented_data['question']
            if j < len(ai_answers_for_batch):
                ai_answer = ai_answers_for_batch[j]
                correct_answer = question_data['resposta_correta']
                is_valid_ai_answer = not ai_answer.startswith("Erro") and not ai_answer.startswith("Bloqueado")
                is_correct = is_valid_ai_answer and (ai_answer == correct_answer)
                results.append({'id': question_data['id'], 'enunciado': question_data['enunciado'],
                                'resposta_ia': ai_answer, 'resposta_correta': correct_answer,
                                'resultado': 'CERTO' if is_correct else ('ERRADO' if is_valid_ai_answer else 'ERRO_IA'),
                                'areas_medicas': ", ".join(question_data['areas_medicas']),
                                'continha_imagem': question_data['continha_imagem']})
            else:
                 print(f"Erro: Resposta ausente para questão {question_data['id']} no lote.")
                 results.append({'id': question_data['id'], 'enunciado': question_data['enunciado'],
                                'resposta_ia': 'Erro: Resposta Ausente no Lote',
                                'resposta_correta': question_data['resposta_correta'], 'resultado': 'ERRO_IA',
                                'areas_medicas': ", ".join(question_data['areas_medicas']),
                                'continha_imagem': question_data['continha_imagem']})
        if i + BATCH_SIZE < total_questions:
            print(f"--- Fim do Lote RAG. Aguardando {DELAY_BETWEEN_BATCHES} segundos... ---")
            time.sleep(DELAY_BETWEEN_BATCHES)
    print("\nAvaliação RAG de todos os lotes concluída.")
    return results

# --- Funções de Cálculo de Acurácia e Salvamento (sem alterações) ---
def calculate_accuracy(results, areas_of_interest):
    total_processed = len(results); total_correct = 0
    area_counts = defaultdict(lambda: {'total': 0, 'correct': 0, 'errors': 0})
    errors_parsing, errors_api, errors_blocked, errors_total_ia = 0, 0, 0, 0
    for result in results:
        if result['resultado'] == 'ERRO_IA':
            errors_total_ia += 1
            if "Parsing" in result['resposta_ia'] or "Ausente" in result['resposta_ia']: errors_parsing += 1
            elif "API" in result['resposta_ia'] or "Erro:" in result['resposta_ia'] or "Vazia" in result['resposta_ia']: errors_api += 1
            elif "Bloqueado" in result['resposta_ia']: errors_blocked += 1
        elif result['resultado'] == 'CERTO': total_correct += 1
        question_areas = result['areas_medicas'].split(', '); processed_for_area_calc = set()
        for area in question_areas:
            if area in areas_of_interest and area not in processed_for_area_calc:
                area_counts[area]['total'] += 1
                if result['resultado'] == 'CERTO': area_counts[area]['correct'] += 1
                elif result['resultado'] == 'ERRO_IA': area_counts[area]['errors'] += 1
                processed_for_area_calc.add(area)
    overall_accuracy = (total_correct / total_processed * 100) if total_processed > 0 else 0
    valid_processed_for_accuracy = total_processed - errors_total_ia
    accuracy_excluding_errors = (total_correct / valid_processed_for_accuracy * 100) if valid_processed_for_accuracy > 0 else 0
    area_accuracy = {}
    for area, counts in area_counts.items():
        valid_area_processed = counts['total'] - counts['errors']
        area_accuracy[area] = (counts['correct'] / valid_area_processed * 100) if valid_area_processed > 0 else 0
    summary = {'total_questions_processed': total_processed, 'total_correct': total_correct,
               'overall_accuracy': overall_accuracy, 'accuracy_excluding_errors': accuracy_excluding_errors,
               'area_accuracy': area_accuracy, 'area_counts': area_counts, 'errors_parsing': errors_parsing,
               'errors_api': errors_api, 'errors_blocked': errors_blocked, 'errors_total_ia': errors_total_ia,
               'valid_processed_for_accuracy': valid_processed_for_accuracy}
    return summary

def save_results_to_csv(results, filepath):
    if not results: print("Nenhum resultado para salvar."); return
    try:
        df = pd.DataFrame(results)
        cols = ['id', 'enunciado', 'resposta_ia', 'resposta_correta', 'resultado', 'areas_medicas', 'continha_imagem']
        df = df.reindex(columns=cols, fill_value='')
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f"Resultados salvos com sucesso em '{filepath}' no Google Drive.")
    except IOError: print(f"Erro ao escrever CSV '{filepath}'. Verifique permissões.")
    except Exception as e: print(f"Erro inesperado ao salvar CSV: {e}")

# --- Nova Função para Salvar Log ---
def save_summary_log(log_filepath, summary, rag_info):
    """Salva um resumo da execução em um arquivo de log de texto."""
    try:
        with open(log_filepath, 'w', encoding='utf-8') as f:
            f.write("--- Log de Execução da Avaliação de IA ---\n")
            f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Arquivo de Questões: {JSON_FILE_PATH}\n")
            f.write(f"Modelo IA: {GEMINI_MODEL_NAME}\n")
            f.write(f"Temperatura IA: {AI_TEMPERATURE}\n")
            f.write(f"Tamanho do Lote (Batch Size): {BATCH_SIZE}\n")
            f.write(f"Máximo de Questões Processadas: {summary['total_questions_processed']} (Limite configurado: {MAX_QUESTIONS_TO_PROCESS})\n")
            f.write(f"Áreas Filtradas: {', '.join(AREAS_TO_FILTER)}\n")
            f.write("\n--- Fonte de Conhecimento RAG ---\n")
            f.write(rag_info + "\n") # Informação sobre PDFs ou índice carregado
            f.write(f"Chunks: {len(pdf_text_chunks) if pdf_text_chunks else 'N/A'}\n")
            f.write(f"Top K Recuperação: {TOP_K_RAG}\n")
            f.write("\n--- Resumo dos Resultados ---\n")
            f.write(f"Total de Acertos: {summary['total_correct']}\n")
            f.write(f"Total de Erros da IA: {summary['errors_total_ia']}\n")
            f.write(f"  - Erros de Parsing/Ausente: {summary['errors_parsing']}\n")
            f.write(f"  - Erros de API/Outros: {summary['errors_api']}\n")
            f.write(f"  - Respostas Bloqueadas: {summary['errors_blocked']}\n")
            f.write(f"Questões Válidas para Acurácia: {summary['valid_processed_for_accuracy']}\n")
            f.write(f"Acurácia Geral (sobre todas): {summary['overall_accuracy']:.2f}%\n")
            f.write(f"Acurácia (excluindo erros IA): {summary['accuracy_excluding_errors']:.2f}%\n")
            f.write("\nAcurácia por Área Médica (sobre válidas):\n")
            if summary['area_counts']:
                for area in AREAS_TO_FILTER:
                     if area in summary['area_counts']:
                         counts = summary['area_counts'][area]
                         acc = summary['area_accuracy'].get(area, 0)
                         valid_area_proc = counts['total'] - counts['errors']
                         f.write(f"  - {area}: {acc:.2f}% ({counts['correct']} de {valid_area_proc} válidas / {counts['total']} total / {counts['errors']} erros IA)\n")
                     else: f.write(f"  - {area}: Nenhuma questão processada.\n")
            else: f.write("  Nenhuma questão encontrada para as áreas.\n")
            f.write("\n--- Fim do Log ---\n")
        print(f"Log de resumo salvo com sucesso em '{log_filepath}'")
    except IOError:
        print(f"Erro: Não foi possível escrever no arquivo de log '{log_filepath}'. Verifique permissões.")
    except Exception as e:
        print(f"Erro inesperado ao salvar log: {e}")


# @title 6. Setup RAG (Carregar PDFs ou Índice Salvo) - Execute esta célula UMA VEZ por sessão ou se os PDFs mudarem

# --- Variáveis Globais para RAG ---
pdf_text_chunks = None
faiss_index = None
rag_source_info = "Nenhuma fonte RAG processada ainda." # Para o log
processed_pdf_files = [] # Para o log
# ---------------------------------

print("--- Iniciando Setup RAG (com Persistência) ---")

# Tenta carregar o índice e os chunks salvos
try:
    if os.path.exists(FAISS_INDEX_PATH) and os.path.exists(TEXT_CHUNKS_PATH):
        print(f"Carregando índice FAISS de {FAISS_INDEX_PATH}...")
        faiss_index = faiss.read_index(FAISS_INDEX_PATH)
        print(f"Carregando chunks de texto de {TEXT_CHUNKS_PATH}...")
        with open(TEXT_CHUNKS_PATH, 'rb') as f:
            pdf_text_chunks = pickle.load(f)
        if faiss_index is not None and pdf_text_chunks is not None and len(pdf_text_chunks) > 0 and faiss_index.ntotal == len(pdf_text_chunks):
             print(f"Índice ({faiss_index.ntotal} vetores) e Chunks ({len(pdf_text_chunks)}) carregados com sucesso.")
             rag_source_info = f"Índice e chunks carregados de {INDEXES_DIR}" # Atualiza info para log
             print("--- Setup RAG Concluído (a partir de arquivos salvos) ---")
        else:
             print("Arquivos de índice/chunks encontrados, mas inválidos ou inconsistentes. Recriando...")
             pdf_text_chunks = None; faiss_index = None
             raise FileNotFoundError("Inconsistência nos arquivos salvos.")
    else:
        print("Arquivos de índice/chunks não encontrados. Iniciando criação do zero...")
        raise FileNotFoundError("Arquivos de índice/chunks não encontrados.")

except (FileNotFoundError, EOFError, pickle.UnpicklingError, Exception) as load_error:
    print(f"Não foi possível carregar arquivos salvos ou eles não existem/são inválidos ({load_error}). Criando RAG do zero...")
    pdf_text_chunks = None
    faiss_index = None
    processed_pdf_files = [] # Reseta a lista de arquivos

    # 1. Carregar e dividir PDFs do diretório
    pdf_text_chunks, processed_pdf_files = load_and_split_pdfs_from_directory(KNOWLEDGE_BASE_DIR)

    if pdf_text_chunks:
        # 2. Gerar Embeddings
        pdf_embeddings = generate_embeddings(pdf_text_chunks)
        if pdf_embeddings is not None and len(pdf_embeddings) == len(pdf_text_chunks):
            # 3. Criar Índice FAISS
            faiss_index = create_faiss_index(pdf_embeddings)
            if faiss_index:
                # 4. Salvar o índice e os chunks
                try:
                    os.makedirs(os.path.dirname(FAISS_INDEX_PATH), exist_ok=True)
                    print(f"Salvando índice FAISS em {FAISS_INDEX_PATH}...")
                    faiss.write_index(faiss_index, FAISS_INDEX_PATH)
                    print(f"Salvando chunks de texto em {TEXT_CHUNKS_PATH}...")
                    with open(TEXT_CHUNKS_PATH, 'wb') as f: pickle.dump(pdf_text_chunks, f)
                    print("Índice e Chunks salvos com sucesso.")
                    rag_source_info = f"Índice criado a partir dos PDFs: {', '.join(processed_pdf_files)}" # Atualiza info para log
                    print("--- Setup RAG Concluído (criado do zero e salvo) ---")
                except Exception as save_error:
                    print(f"Erro ao salvar índice/chunks: {save_error}. RAG funcionará nesta sessão, mas precisará ser recriado.")
                    rag_source_info = f"Índice criado (erro ao salvar) a partir dos PDFs: {', '.join(processed_pdf_files)}"
            else:
                print("--- Falha ao criar índice FAISS ---")
                pdf_text_chunks = None; rag_source_info = "Falha ao criar índice FAISS."
        else:
            print(f"--- Falha ao gerar embeddings. ---")
            pdf_text_chunks = None; rag_source_info = "Falha ao gerar embeddings."
    else:
        print(f"--- Falha ao carregar/processar PDFs de {KNOWLEDGE_BASE_DIR}. O RAG não será utilizado. ---")
        rag_source_info = f"Falha ao carregar PDFs de {KNOWLEDGE_BASE_DIR}."


# @title 7. Execução Principal da Avaliação com RAG
if __name__ == "__main__":
    print("\n--- Iniciando Script de Avaliação com RAG (Múltiplos PDFs e Persistência) ---")

    # Gera o timestamp para os nomes dos arquivos de saída
    timestamp_str = datetime.now().strftime("%Y-%m-%d-%H-%M")
    output_csv_path_ts = os.path.join(RESULTS_DIR, f"{BASE_CSV_FILENAME}_{timestamp_str}.csv")
    output_log_path_ts = os.path.join(RESULTS_DIR, f"{BASE_LOG_FILENAME}_{timestamp_str}.txt")

    if 'API_KEY' not in locals() or not API_KEY:
        print("!!! ERRO CRÍTICO: API Key não configurada. Interrompendo. !!!")
    elif faiss_index is None or pdf_text_chunks is None:
         print("!!! ERRO CRÍTICO: Setup RAG não foi concluído com sucesso. Execute a célula '6. Setup RAG' primeiro. Interrompendo. !!!")
    else:
        print("\nCarregando dados do JSON...")
        all_data = load_json_data(JSON_FILE_PATH)

        if all_data:
            print(f"Total de questões no arquivo: {len(all_data)}")
            print(f"Filtrando por áreas: {', '.join(AREAS_TO_FILTER)}...")
            filtered_data = filter_questions(all_data, AREAS_TO_FILTER)
            print(f"Número de questões filtradas: {len(filtered_data)}")

            if not filtered_data:
                print("Nenhuma questão encontrada para as áreas especificadas.")
            else:
                questions_to_process = filtered_data
                # Limita o número de questões se necessário
                if MAX_QUESTIONS_TO_PROCESS is not None and MAX_QUESTIONS_TO_PROCESS > 0 and MAX_QUESTIONS_TO_PROCESS < len(filtered_data):
                    print(f"Limitando o processamento às primeiras {MAX_QUESTIONS_TO_PROCESS} questões filtradas.")
                    questions_to_process = filtered_data[:MAX_QUESTIONS_TO_PROCESS]
                elif MAX_QUESTIONS_TO_PROCESS is not None and MAX_QUESTIONS_TO_PROCESS > 0:
                     print(f"Processando todas as {len(filtered_data)} questões filtradas.")
                elif MAX_QUESTIONS_TO_PROCESS == 0:
                    print("MAX_QUESTIONS_TO_PROCESS definido como 0. Nenhuma questão será processada.")
                    questions_to_process = []
                else: # MAX_QUESTIONS_TO_PROCESS é None
                    print(f"Processando todas as {len(filtered_data)} questões filtradas.")

                if questions_to_process:
                    # Avalia as questões USANDO RAG
                    evaluation_results = evaluate_questions_with_rag(questions_to_process, faiss_index, pdf_text_chunks)

                    if evaluation_results:
                        accuracy_summary = calculate_accuracy(evaluation_results, AREAS_TO_FILTER)
                        # Salva CSV e Log com timestamp
                        save_results_to_csv(evaluation_results, output_csv_path_ts)
                        save_summary_log(output_log_path_ts, accuracy_summary, rag_source_info)

                        # Mostra o resumo da acurácia (igual ao log)
                        print("\n--- Resumo da Avaliação (com RAG de Múltiplos PDFs) ---")
                        print(f"Modelo IA Utilizado: {GEMINI_MODEL_NAME}")
                        print(f"Temperatura Utilizada: {AI_TEMPERATURE}")
                        print(f"Fonte de Conhecimento RAG: {rag_source_info}")
                        print(f"Total de Questões Processadas: {accuracy_summary['total_questions_processed']}")
                        print(f"Total de Acertos: {accuracy_summary['total_correct']}")
                        print(f"Total de Erros da IA: {accuracy_summary['errors_total_ia']}")
                        print(f"  - Erros de Parsing/Ausente: {accuracy_summary['errors_parsing']}")
                        print(f"  - Erros de API/Outros: {accuracy_summary['errors_api']}")
                        print(f"  - Respostas Bloqueadas: {accuracy_summary['errors_blocked']}")
                        print(f"Questões Válidas para Acurácia: {accuracy_summary['valid_processed_for_accuracy']}")
                        print(f"\nAcurácia Geral (sobre todas): {accuracy_summary['overall_accuracy']:.2f}%")
                        print(f"Acurácia (excluindo erros IA): {accuracy_summary['accuracy_excluding_errors']:.2f}%")
                        print("\nAcurácia por Área Médica (sobre válidas):")
                        if accuracy_summary['area_counts']:
                            for area in AREAS_TO_FILTER:
                                 if area in accuracy_summary['area_counts']:
                                     counts = accuracy_summary['area_counts'][area]
                                     acc = accuracy_summary['area_accuracy'].get(area, 0)
                                     valid_area_proc = counts['total'] - counts['errors']
                                     print(f"  - {area}: {acc:.2f}% ({counts['correct']} de {valid_area_proc} válidas / {counts['total']} total / {counts['errors']} erros IA)")
                                 else: print(f"  - {area}: Nenhuma questão processada.")
                        else: print("  Nenhuma questão encontrada para as áreas.")
                        print(f"\nResultados detalhados salvos em: {output_csv_path_ts}")
                        print(f"Log de resumo salvo em: {output_log_path_ts}")
                    else: print("Avaliação RAG não produziu resultados.")
                else: print("Nenhuma questão selecionada para processamento.")
        else: print("Não foi possível carregar os dados do JSON.")

    print("\n--- Fim do Script ---")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado com sucesso!
Diretório de índices '/content/drive/MyDrive/Indexes/' verificado/criado.
Diretório de resultados '/content/drive/MyDrive/Resultados/' verificado/criado.
API Key do Google configurada com sucesso via Colab Secrets.
--- Iniciando Setup RAG (com Persistência) ---
Carregando índice FAISS de /content/drive/MyDrive/Indexes/medicina_knowledge.index...
Carregando chunks de texto de /content/drive/MyDrive/Indexes/medicina_knowledge_chunks.pkl...
Índice (359 vetores) e Chunks (359) carregados com sucesso.
--- Setup RAG Concluído (a partir de arquivos salvos) ---

--- Iniciando Script de Avaliação com RAG (Múltiplos PDFs e Persistência) ---

Carregando dados do JSON...
Total de questões no arquivo: 670
Filtrando por áreas: Ginecologia, Obstetrícia, Pediatria...
Número de questões filtradas: 209
Limitando o processamento às primeiras 15